# Objectives

* Define pressure and density.
* State the density of fresh water.
* State the value of standard atmospheric pressure in SI units.
* Explain Pascal's principle.
* Explain the relationship between pressure and depth in a fluid.
* Solve for the column height of a fluid in a U-tube using Pascal's principle.
* State Archimedes' Principle.
* Calculate the buoyant force on submerged or floating objects.

# Fluids

A **fluid** is a substance that can readily flow from place to place and take on the shape of a container. Fluid can be a liquid or a gas.

For any type of object, fluid or otherwise, we can define the **density** of the object. It has a symbol $\rho$ ("rho") and is defined by mass over volume, or

$$
    \rho = \frac{M}{V}
$$

The density has SI units of kg/m$^3$. *You should memorize the density of fresh water as 1000 kg/m$^3$.*

> **Problem:** A cube of metal has a mass of 347 g and measures 3.21 cm on a side. Using the table of densities given, identify the metal.
> 
> | Substance | $\rho$ (kg/m$^3$) |
> |-----------|-------------------|
> | aluminum  | 2700              |
> | iron      | 7860              |
> | silver    | 10500             |
> | lead      | 11300             |
> | mercury   | 13600             |
> | gold      | 19300             |

Historically, one of the more important ideas in physics from the 18th and 19th centuries is the [kinetic theory of gases](https://en.wikipedia.org/wiki/Kinetic_theory_of_gases), which explains many of the properties of a gas by the simple model of assuming the gas is made of very small particles moving at random speeds. We will explore this idea a little as we motivate how the pressure of a fluid can change based on the depth the pressure is measured at. A good simulation to explore this model, and to review how it accounts for the various gas laws, is given in the next cell.

In [ ]:
from IPython.display import IFrame

IFrame(src="https://phet.colorado.edu/sims/html/gases-intro/latest/gases-intro_en.html", width="600", height="400")

We will now construct our own simple vPython simulation of a gas. First, we need to import all the necessary vPython modules.

In [5]:
from vpython import canvas, cos, curve, cylinder, dot, mag, norm, random, pi, rate, sin, sqrt, vector

<IPython.core.display.Javascript object>

To keep things visually simple, we will use a two-dimensional gas, so you can see everything that is happening at all times. This means we will have disks representing the molecules of the gas inside of a square box. There are several things we need to do to set up this simulation:

* Create the disks inside the box with an initial velocity
* Check for collisions between each pair of disks
* Check for collisions between the disks and the walls

In order to not lose the forest because of the trees, I will create each of these tasks as a separate function in vPython. The first creates the disks -- the basic idea is to create the disks at random locations. The function returns three lists: the list `disk_pos_list` of disk positions, a list `disk_vel_list` of their velocities, and then a list `disk_object_list` of the `cylinder` objects representing them in the animation. Since this is a relatively simple way of setting up the simulation, to make sure disks do not form on top of each other, the ratio `AREA_RATIO` of the area of the disks to the total area is kept small. The positions and velocities of the disks are also kept in a Python list, rather than directly creating the `cylinder` objects that will represent the atoms in our simulation. Using a list will help the animation run more smoothly. Run the cell below to use this function.

In [6]:
# Create disks

def createDisks():
    
    # Create disk radius as global variable, so that all other
    # functions can know about it
    
    global R
    
    # Set disk size. We want to make it easier to find a random list
    # of positions for the disks, so we must make sure that the area
    # of all the disks is less than the total area of the square. To
    # do this, we assume
    #
    #        (NUM_DISKS * pi * R^2) / (2 * L)^2 << 1
    #
    # Specficially, the ratio is set to the value AREA_RATIO, and the
    # resulting radius R is solved for.

    AREA_RATIO = 0.1
    R = L * sqrt(AREA_RATIO / (NUM_DISKS * pi))
    
    # Initial speed of all disks
    
    INIT_SPEED = 5
                    
    # Create list of disk objects
    
    disk_object_list = []
        
    for iii in range(NUM_DISKS):

        # Random position; the center of the disk must be
        # at least one disk radius away from box edge
        
        x = (L - R) * (1 - 2 * random())
        y = (L - R) * (1 - 2 * random())
        z = 0
        
        # Random direction for velocity; all disk speeds
        # are set to be the same value INIT_SPEED
        
        Q = 2 * pi * random()
        vx = INIT_SPEED * cos(Q)
        vy = INIT_SPEED * sin(Q)
        vz = 0
        
        # Put these into disk object list
        
        disk_object_list = disk_object_list + [cylinder(pos = vector(x, y, z), \
                            velocity = vector(vx, vy, vz), axis = vector(0, 0, 0.01), radius = R)]

    # Send list back to main procedure
    
    return disk_object_list

Next is a function to check whether the disks have hit the walls. This is relatively simple, and we have used something similar in previous simulations. Since the sides of the box run from `-L` to `L` in both the $x$ and $y$ directions, we just have to check whether the disk center `disk_pos` has gone *beyond* one disk radius `R` from the walls. This is the same as making sure that

$$
    |x| >= (L - R) \qquad
    |y| >= (L - R)
$$

Verify this for each wall if you are unsure of this!

In [7]:
# Check for collisions against wall

def checkWallCollision(disk):
    
    # Check to see if the disk is too close to the walls;
    # flip the direction of the appropriate velocity
    # component if this does happen
    
    if abs(disk.pos.x) >= (L - R):
        disk.velocity.x = -disk.velocity.x
    if abs(disk.pos.y) >= (L - R):
        disk.velocity.y = -disk.velocity.y 
        
    # If a disk has gotten loose, use modular arithmetic
    # to put it back in the box on the inside
        
    if abs(disk.pos.x) > L:
        disk.pos.x = (disk.pos.x + 2 * L) % L
        
    if abs(disk.pos.y) > L:
        disk.pos.y = (disk.pos.y + 2 * L) % L
        
    # Return disk object with possibly changed
    # position, velocity vectors
        
    return disk

Finally, our third function checks for collisions between a pair of disks. Remember when we considered collisions in Lesson LESSON LINK, that we use conservation of momentum to find the final velocities of the two disks after the collision. This is done in three steps:

* From the disk velocities that we see, use relative motion find the velocities of the disks in the center of mass (CM) frame.
* Use conservation of momentum in the CM frame. The collision is assumed to be elastic, since the disks are "hard", so the coefficient of restitution is $\epsilon = 1$. This means that the CM frame velocities for disks 1 and 2 are

$$
    {\vec v}_{1f, CM} = - {\vec v}_{1i, CM} \qquad
    {\vec v}_{2f, CM} = - {\vec v}_{2i, CM}
$$

* Take the final CM velocities and use relative motion again to find the final velocities that we see. Return these velocities as `disk1_vel` and `disk2_vel`.

In [8]:
# Check for collisions between pairs of disks

def checkDiskCollision(disk1, disk2):
    
    # Collision occurs if the difference between the positions
    # is less than twice the individual disk radius
    
    if mag(disk1.pos - disk2.pos) <= 2 * R:
        
        # Transfer from lab frame to CM frame
        
        CM_vel = 0.5 * (disk1.velocity + disk2.velocity)
        disk1_CM_vel = disk1.velocity - CM_vel
        disk2_CM_vel = disk2.velocity - CM_vel
        
        # Using initial velocities and Newton's 2nd law in
        # momentum terms, find final velocities of disks
        
        distUnitVec = norm(disk1.pos - disk2.pos)
        disk1_CM_vel = disk1_CM_vel - 2 * dot(disk1_CM_vel, distUnitVec) * distUnitVec
        disk2_CM_vel = disk2_CM_vel - 2 * dot(disk2_CM_vel, distUnitVec) * distUnitVec
        
        # Transfer from CM frame back to lab frame
        
        disk1.velocity = disk1_CM_vel + CM_vel
        disk2.velocity = disk2_CM_vel + CM_vel
        
    return [disk1, disk2]

Now, we can put this all together into our main procedure, which is given in the cell below. Run the cell, and you should see our gas of disks merrily moving around a box, colliding with each other and the walls.

In [13]:
canvas()

# Constants

L = 2
NUM_DISKS = 20
MAX_TIME = 5
t = 0
dt = 0.01

# Create disks with random starting positions, velocities

diskList = createDisks()

# Create boundary of square for disks

boundary = curve(radius = 0.01)
boundary.append(vector(L, L, 0), vector(L, -L, 0), vector(-L, -L, 0), vector(-L, L, 0), vector(L, L, 0))

# Time evolution

while t < MAX_TIME:
    rate(100)
    
    # Go through all the disks, check for collisions, and
    # then update the position, velocity of the disk
    
    for jjj in range(NUM_DISKS):
    
        # Check for wall collisions
    
        diskList[jjj] = checkWallCollision(diskList[jjj])
        
        # Check for disk collisions; to save computational
        # time, only check for collisions with disks having
        # numerical labels greater than the current disk
        
        for kkk in range(jjj + 1, NUM_DISKS):
            [diskList[jjj], diskList[kkk]] = checkDiskCollision(diskList[jjj], diskList[kkk])
        
        # Update positions, velocities in lists
        
        diskList[jjj].pos = diskList[jjj].pos + diskList[jjj].velocity * dt
        
    # Update time
    
    t = t + dt

<IPython.core.display.Javascript object>

Sometimes, if you run many simulations in the same Jupyter notebook, your computer may slow down. This is partly because the computer may keep track of "old" vPython objects -- such as the disks in our gas simulation -- even though they are no longer being displayed. The cell below acts as a measure to remove previous disks from memory. If you run it, you will see the disks in the `canvas` above disappear!

In [14]:
# For any object 'thing', make
# it invisible and delete it

def cleanup(thing):
    try:
        thing.visible = False
        del thing
    except:
        pass

In [ ]:
# Run through the disk objects
# of the simulation and clean
# them up using above procedure
    
for disk in diskList:
    cleanup(disk)

The vPython script run in the cell below uses various methods to try and speed up calculations for the disk gas. Thus, it can run roughly the same speed with a greater number of disks.

In [12]:
%run ../scripts/2D_disk_gas.py

<IPython.core.display.Javascript object>

In [15]:
# Clean up disks from script
    
for disk in diskObjList:
    cleanup(disk)

# Pressure

## Definition of pressure

Now that we have a simple model of a fluid, let's use it to get an intuition for how fluids generate pressure. Consider what happens when you push both a pin and the blunt end of a pen against your skin with the same force. What will determine whether your skin will be punctured?

(MORE)

>**Pressure:**
>
> * Symbol: $P$
> * Definition:
> $$
    P = \frac{F}{A}
 $$
> * Units: Pascals
> $$
    1\ \textrm{Pa} = 1\ \frac{\textrm{N}}{\textrm{m}^2}
 $$

For example, squeezing an egg from all sides will *not* break the egg. The curved shape of the egg is able to redistribute the force all across its area, so the resulting pressure on the egg is too small to break the shell. However, exerting a force on a small area -- by hitting the egg against the edge of a bowl -- results in a large enough pressure to crack the egg!

> **Problem:** You are walking out on a frozen lake and you begin to hear the ice cracking beneath you. What is your best strategy for getting off the ice safely?
>
> 1. Jump up and down to lessen your contact time with the ice.
> 1. Lie down flat on the ice and crawl toward shore.
> 1. Shuffle your feet (without lifting them) to move toward shore.
> 1. Stand absolutely still and wait for rescue.

![A tire gauge is used to measure the difference in the pressure inside the tire *above* atmospheric pressure](../img/INT_AY20_MP2_L06_Fig01-Tire_gauge_small.png)

There are other non-SI units that you will see when dealing with pressure. One unit is the *atmosphere* (atm), which is the standard average pressure at sea level on the Earth's surface. We will deal more with atmospheric pressure in Section SECTION LINK. Another such unit is *pounds per square inch* (psi). The conversion factors between these units are listed below.
    
* Atmospheres -- 1 atm = 101.5 kPa
* 1 atm = 14.7 $\frac{\text{lb}}{\text{in}^2}$ (psi)

Most tire gauges measure the pressure of vehicle tires in psi. You might also see units of *mm of Hg* (Torr). Torr are most often used when measuring blood pressure or by meteorologists discussing the weather!

![Two types of heel (high and flat) give different pressures on the foam beneath them](../img/INT_AY20_MP2_L06_Fig02-Shoes_on_foam_small.png)

> **Problem:** A 60.0 kg woman is standing up such that the weight is distributed equally on both feet. If her heels are supporting 75.0 percent of her weight and the contact area between her heels and the ground are 0.250 in$^2$ (for the high heel) and 2.00 in$^2$ (for the flat heel), what is the pressure value on each heel?

## Atmospheric pressure

Atmospheric pressure is the pressure due to the weight of the air above.
* Notation: $P_{at}$ -- atmospheric pressure; the **standard** value is
$$
    P_{at} = 1\ \textrm{atm} = 101.3 \ \textrm{kPa} = 14.7\ \frac{\text{lb}}{\text{in}^2}
$$
* If a problem does not give the atmospheric pressure, you may assume a standard pressure of 101.3 kPa.

![A Goethe barometer, originally developed by the German writer Johann Wolfgang Von Goethe ([source](https://commons.wikimedia.org/wiki/File:Barometer_Goethe_01.jpg))](../img/Barometer.png)

A barometer is formed when a tube filled with fluid and blocked at both ends is put into a basin of the same fluid, and the bottom end is uncapped. The fluid level in the top of the tube will go down to some level and stay. Why is this? 

![Portrait of Evangelista Torricelli ([source](https://commons.wikimedia.org/wiki/File:Evangelista_Torricelli_by_Lorenzo_Lippi_(circa_1647,_Galleria_Silvano_Lodi_%26_Due).jpg))](../img/INT_AY20_MP2_L06_Fig03-Evangelista_Torricelli_small.png)

Based on Aristotle's ideas -- "nature abhors a vacuum" -- it was originally believed that a vacuum was formed and could pull the fluid up with a limited amount of force. However, this idea was tested by various people in the 1630s and 1640s. Evangelista Torricelli thought instead it was the weight of the air itself *pushing* the fluid up, until the forces from the air and fluid weights balanced.

![Portrait of Blaise Pascal ([source](https://commons.wikimedia.org/wiki/File:Blaise_Pascal_Versailles.JPG))](../img/INT_AY20_MP2_L06_Fig04-Blaise_Pascal.png)

This idea was tested by Blaise Pascal in various ways in 1644, and it was shown that Torricelli was correct.

Gauge pressure is the difference between total pressure and atmospheric pressure

$$ 
    P_g = P_{tot} - P_{at}
$$

where $P_g$ is gauge pressure, $P_{tot}$ is total pressure and $P_{at}$ is atmospheric pressure. Tire gauges and other measuring devices give the pressure of the tire *above* the atmospheric pressure, hence the name.

> **Problem:** A glass bottle of soda is sealed with a screw cap. The absolute pressure of the carbon dioxide inside the bottle is 160 kPa. Assuming the top and bottom surfaces of the cap each have an area of 4.74 cm$^2$, obtain the magnitude of the force (in N) the screw thread exerts on the cap in order to keep it on the bottle. The air pressure outside the bottle is one atmosphere.

![2019 Chevrolet Corvette ZR1 at the Montreal International Car Show ([source](https://commons.wikimedia.org/wiki/File:2019_Chevrolet_Corvette_ZR1_au_SIAM_2019.jpg))](../img/2019_Chevrolet_Corvette.png)

> **Problem:** The weight of your 1420 kg car (2019 Corvette ZR1) is supported equally by its four tires, each inflated to a gauge pressure of 35.0 psi.  What is the area of contact each tire makes with the road (in cm$^2$)?

# Pressure at a depth

![Pressure at a depth in a fluid](../img/INT_AY20_MP2_L07_Fig01-Pressure_at_a_depth_small.png)

$$
    P_{tot} = P_{at} + \rho g h
$$

* $P_{tot}$ -- total pressure at depth $h$ below the surface 
* $P_{at}$ -- total pressure at the surface of the fluid (typically atmospheric pressure is used)
* $\rho$ -- density of fluid
* $g$ -- magnitude of acceleration due to gravity (9.81 m/s$^2$ on Earth's surface)
* $h$ -- depth below the surface of the fluid
                                                   
(DO A BETTER JOB OF SAYING THIS IS SPECIFIC FOR ONE FLUID, AND TALK ABOUT GENERAL CASE)

In [ ]:
from IPython.display import IFrame

IFrame(src="https://phet.colorado.edu/sims/html/under-pressure/latest/under-pressure_en.html", width="600", height="400")

> **Problem:** The figure below shows four identical boxes held stationary in a pool of water.  Which choice below correctly ranks the fluid pressure on the bottom of each block, from greatest to least?
>
> ![](../img/INT_AY20_MP2_L07_Fig02-Rank_the_boxes_small.png)

> **Problem:** A swimmer starts at the surface of the ocean and descends 30.0 feet. What is the total pressure 30.0 feet below the surface of the ocean (in atm)? Assume the density of sea water is 1025 kg/m$^3$.

> **Problem:** The *Titanic* was found in 1985 lying on the bottom of the North Atlantic at a depth of 2.50 miles. What is the total pressure at this depth? Note: $\rho = 1025$ kg/m$^3$.
>
> ![RMS Titanic departing Southampton on April 10, 1912 ([source](https://commons.wikimedia.org/wiki/File:RMS_Titanic_3.jpg))](../img/RMS_Titanic.png)

> **Problem:** Water in the lake behind Hoover Dam is 221 m deep. What is the *water pressure* (aka gauge pressure) at the base of the dam?
>
> ![Hoover dam seen from the air ([source](https://commons.wikimedia.org/wiki/File:Hoover_dam_from_air.jpg))](../img/Hoover_dam.png)

> **Problem:** The Mariana Trench is located in the Pacific Ocean at a depth of about 11.7 km below the surface of the water. The density of sea water is 1025 kg/m$^3$. If an underwater vehicle were to explore such a depth, what force (in MN) would the water exert on the vehicle's observation window, with a radius of 13.0 cm?
>
> ![The U.S. Navy bathyscaphe *Trieste*, just before her record dive to the bottom of the Marianas Trench, 23 January 1960. The waves were were about 1,5 to 1,8 m high when scientist Jacques Piccard and Lt. Don Walsh, USN, and Piccard boarded from the rubber raft attached to the bathyscaphe. The destroyer escort USS *Lewis* (DE-535) is steaming by in the background. ([source](https://commons.wikimedia.org/wiki/File:Hoover_dam_from_air.jpg))](../img/Bathyscaphe_Trieste.png)

# Pascal's principle

An external pressure applied to an enclosed fluid is transmitted unchanged to every point in the fluid. Pressure of the enclosed fluid at one vertical height is the same pressure at a different point with the same vertical height.

![Pascal's principle says the pressure throughout the pool changes due to an external change in pressure](../img/INT_AY20_MP2_L06_Fig05-Diver_in_pool.png)

> **Problem:** If the density of oil is 800. kg/m$^3$, what is the height $h$ (in cm) of the column of oil shown to the right?
>
>
> ![U-tube filled with water](../img/INT_AY20_MP2_L07_Fig03-U_tube_small.png)

> **Problem:** A U-shaped tube is filled mostly with fresh water, but a small amount of vegetable oil has been added to both sides. Calculate the density of the oil in kg/m$^3$, using the distances $h_1 = 4.00$ cm, $h_2 = 9.00$ cm, and $h_3 = 3.00$ cm.
>
>
> ![U-tube filled with water and vegetable oil](../img/INT_AY20_MP2_L07_Fig04-U_tube_small.png)

Torpedoes do not sink ships by holing their hulls, but instead are fired underneath the keel of the target. When the torpedo detonates, it creates a pressure wave, which forms the boundary of a steam bubble, lifting the hull and weakening the keel. The steam bubble quickly collapses, dropping the ship downward and breaking the keel. For more information, see the chapter [Underwater Weapons](https://fas.org/man/dod-101/navy/docs/es310/uw_wpns/uw_wpns.htm) at the on-line [Introduction to Naval Weapons](http://fas.org/man/dod-101/navy/docs/es310/syllabus.htm).

![How a torpedo breaks a ship's keel](../img/INT_AY20_MP2_L06_Fig06-Torpedo_effect_on_ship.png)

In June 1999, the Australian submarine HMAS Farncomb conducted a live-fire sinking exercise against the former HMAS Torrens, a destroyer escort. The video linked in the cell below clearly shows the intended effects of a torpedo on a target ship.

In [ ]:
from IPython.display import VimeoVideo

VimeoVideo(130514745, width=600, height=400, muted=1)

# Buoyancy

## Archimedes and the King

* Archimedes was a relative of the King of Syracuse, Hieron
* Hieron had ordered a gold crown, but suspected he had been cheated
* Hieron asked Archimedes to figure out the truth!

![A golden laurel wreath from the Hellenstic Era ([source](https://commons.wikimedia.org/wiki/File:Golden_laurel_wreath_T_HL_04_Kerameikos_Athens.png))](../img/Wreath.png)

This example is taken from the [Archimedes site](https://www.cs.drexel.edu/~crorres/Archimedes/contents.html)

When you place an object in a fluid, it will displace a volume of fluid equal to the volume of the object in the fluid
$$
    V_{obj.\ in\ fluid} = V_{displ.\ fluid}
$$

![Block displacing fluid when it is immersed in the fluid](../img/INT_AY20_MP2_L08_Fig01-Block_displacing_fluid_small.png)

Fluid is pushed away by the object, so there is a rise in the level of the fluid (only due to object's volume beneath the "water line")

> **Problem:** A wooden block and a beaker of water are placed side-by-side on a scale, which reads weight $F_{g, A}$. The block is then placed into the water, where it floats, and the scale reads $F_{g, B}$; no water spills out of the beaker. How do the two scale readings compare?
>
> 1. $F_{g, A} > F_{g, B}$
> 1. $F_{g, A} = F_{g, B}$
> 1. $F_{g, A} < F_{g, B}$
> 1. Cannot tell without the *density* of the block
> 1. Cannot tell without the *volume* of the block

![Comparing the mass of the crown to that of a piece of gold in *air* ([source](https://www.math.nyu.edu/~crorres/Archimedes/Crown/CrownIntro.html))](../img/crown-air.png)

Find a piece of gold that has the same weight as the crown...

![Comparing the mass of the crown to that of a piece of gold in *water* ([source](https://www.math.nyu.edu/~crorres/Archimedes/Crown/CrownIntro.html))](../img/crown-water.png)

...and then submerge them in water. If the balance tilts, the crown and the gold have different volumes - the crown has the wrong density!



## Archimedes' principle

When you place an object in a fluid, a buoyant force equal to the weight of the displaced fluid acts upward on the object.

![Forces acting on a completely submerged object](../img/INT_AY20_MP2_L08_Fig02-Immersed_block_with_forces_small.png)

The buoyant force $F_b$ is due to the difference in the forces on the top and bottom of the object (see slides on pressure and Section 15-4 in textbook),

$$
	F_b = F_{bottom} - F_{top}
$$

which can be rewritten as

$$
    F_b = F_{g, displ.\ fluid} = \rho_{fl} g V_{sub}
$$

>**Archimedes' principle:** An object immersed (either completely or partially) in a fluid experiences an upward buoyant force equal in magnitude to the weight of fluid displaced by the object.

Buoyant force $F_b = \rho_{fl} g V_{sub}$, where

* $\rho_{fl}$ is the density of the fluid the object is immersed in
* $g$ is the magnitude of the acceleration due to gravity
* $V_{sub}$ is that portion of the object's volume that is immersed in the fluid (which can be the whole volume if the object is completely submerged)

If the density of the object is greater than the density of the fluid, then the object will sink.

> **Problem:** Imagine holding two identical bricks in place under water. Brick 1 is just beneath the surface of the water, while brick 2 is held about two feet down. The force needed to hold brick 2 in place is <u>&emsp;&emsp;&emsp;</u> the force for brick 1.
>
> 1. greater than
> 1. less than
> 1. equal to
>
>![Two immersed blocks](../img/INT_AY20_MP2_L08_Fig03-Two_immersed_blocks_small.png)

> **Problem:**
> * Suppose the crown is measured to have a weight in air of 7.84 N and an apparent weight in water of 6.86 N. What is the mass of the crown?
> * What is the magnitude of the buoyant force on the crown in the water? The buoyant force acts upward in the water, reducing the true weight to an "apparent weight".
> * What is the volume of the crown? Fresh water has a density of 1000 kg/m$^3$. Use Archimedes' principle, finding the volume by computing how much volume of water weighs the same as the buoyant force.
> * What is the density of the crown? Compare to solid gold with a density of 19,300 kg/m$^3$. Using the mass and volume of the crown which we found before, we can compute its density.



## Floating objects

![Forces acting on a floating object](../img/INT_AY20_MP2_L08_Fig04_Floating_block_small.png)

* The floating object is in equilibrium, so forces balance:

$$
    F_b = F_{g, obj.}
$$

* By Archimedes' principle, the buoyant force is equal to the weight of the displaced fluid:

$$
    F_b = F_{g, fluid}
$$

* Putting these equations together gives for a {\bf floating} object that

$$
    F_{g, obj} = F_{g, fluid}
$$

> **Problem:** Two identical blocks are floating in two separate aquariums. Aquarium A is filled with alcohol ($\rho$ = 789 kg/m$^3$), while aquarium B is filled with pure water. Which has the greater buoyant force acting on it?
>
> 1. The block floating in alcohol
> 1. The block floating in pure water
> 1. They both have the same buoyant force
> 1. Cannot determine without the density of the blocks

![The *Occitania* and a 2,500-ton barge carrying a load of coal destined for a thermal power plant ([source](https://commons.wikimedia.org/wiki/File:Barge_%C3%A0_charbon.jpg))](../img/Barge.png)

> **Problem:** You are shopping online for a new barge at [McDonough Marine Services](http://www.mcdonoughmarine.com/ocean-barges.html), and settle on a 140' $\times$ 40' $\times$ 9' 6'' size barge. Below is given the data for the *loadline* and *freeboard* loading for the barge:
>
> <table>
> <tr>
>     <th></th>
>     <th colspan = "4">Metric ton cargo at freeboards of</th>
> </th>
> <tr>
>     <th>Cargo at loadline (mt)</th>
>     <th>3'</th>
>     <th>4'</th>
>     <th>5'</th>
>     <th>6'</th>
> </tr>
> <tr>
>     <td>965</td>
>     <td>820</td>
>     <td>660</td>
>     <td>550</td>
>     <td>340</td>
> </tr>
> </table>
> 
> The **freeboard** is the distance from the deck of the barge down to the water line. The **loadline** is the freeboard when the maximum safe cargo is placed on the barge, so that it is the highest water line for safe operations. A **metric ton** (mt) is 1000 kg.
> 
> * What is the mass of the barge in metric tons? Use a density of 1010 kg/m$^3$ for water.
> * Where is the loadline located on the barge? Give the distance in cm from the barge deck.

![An iceberg in the Arctic Ocean ([source](https://commons.wikimedia.org/wiki/File:Iceberg_in_the_Arctic_with_its_underside_exposed.jpg))](../img/Iceberg.png)

> **Problem:** An iceberg has a density of 917 kg / m$^3$ and is floating in sea water ($\rho = 1025$ kg / m$^3$). What fraction $V_{sub} / V_{tot}$ of the iceberg is submerged in the sea water?

![Object floating in water](../img/INT_AY20_MP2_L08_Fig05-Floating_block_small.png)

> **Problem:** An object floats in water with 3/4 of its volume submerged. What is the ratio of the density of the object to that of water?

![Object floating in oil](../img/INT_AY20_MP2_L08_Fig06-Floating_block_small.png)

> **Problem:** The same object from the previous question is now placed in **oil** with a density half that of water. What happens?

## Ship stability

Put in something here about center of buoyancy versus center of gravity, and how to ensure that a ship is stable to "heeling".

# Summary

* Define pressure, and identify its units (SI and otherwise).
* Use the definition of pressure to find an unknown force or area.
* Define atmospheric and gauge pressures.
* Find the pressure on an object at a certain depth inside of a fluid
* Use Pascal's principle and the relation for pressure at a given depth in a fluid to find the height or density of a fluid in a U-tube.
* Describe the relationship between the portion of an object's volume in a fluid, and the amount of fluid displaced by that object.
* State Archimedes' principle.
* Use Archimedes' principle to find the amount of an object that is immersed in a fluid.